In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

from main import load_and_prepare_sessions
from processing.session_sampling import MiceAnalysis
from config import attr_interval_dict, all_metrics, all_brain_regions, all_event_types
from analysis.timepoint_analysis import sample_low_and_high_signals
from plotting.main_plotting import plot_signals_p_values
import matplotlib.pyplot as plt


sessions = load_and_prepare_sessions("../../../Baseline", load_from_pickle=True, remove_bad_signal_sessions=True)
mouse_analyser = MiceAnalysis(sessions)

In [ ]:
import itertools
import numpy as np
from scipy.stats import ttest_ind
from itertools import chain
from utils import mouse_br_events_count

# brain_reg_to_color = {'LH': 'orange',
#                       'mPFC': 'paleturquoise'}
brain_reg_to_color = {'VS': 'purple',
                      'DMS': 'forestgreen',
                      'DLS': 'C0'}

# Assuming config.all_metrics, config.all_brain_regions, and config.all_events are all iterable
for performance_metric, brain_region, event in itertools.product(all_metrics, all_brain_regions, all_event_types):
    low_signals, high_signals, low_resp_metrics, high_resp_metrics, resp_metric_names = \
        sample_low_and_high_signals('events', performance_metric, brain_region, event, mouse_analyser, n=1000)
    
    sb = attr_interval_dict[event]

    t_stats = np.zeros(low_signals.shape[1])  # Assuming the second dimension is time
    p_values = np.zeros(low_signals.shape[1])
    
    # Perform a t-test at each time point
    for time_point in range(low_signals.shape[1]):
        t_stat, p_value = ttest_ind(low_signals[:, time_point], high_signals[:, time_point])
        t_stats[time_point] = t_stat
        p_values[time_point] = p_value
    
    
    if np.min(p_values) >= 1/3000:
        continue
    suptitle = f"brain regions: {brain_region},  metric: {performance_metric},  event type: {event} ({low_signals.shape[0]})"

    low_subtitle = f"low 3 mice vs. upper 3 mice"
    high_subtitle = ''

    color = brain_reg_to_color[brain_region]
    plot_signals_p_values([low_signals, high_signals], [low_subtitle, high_subtitle], suptitle, color, 
                          5, sb, p_values, fname=None)#f'{performance_metric}_{brain_region}_{event}.png')